In [71]:

import csv
import sys
import collections
import os.path
import requests

# on exit clean-ups
import atexit

# neo4j libs
from py2neo import Graph
from py2neo import Node, Relationship
from py2neo import authenticate
from random import randint

# neo4j graph connector
authenticate("localhost:7474", "neo4j", "1sTep123")
graph = Graph()
# delete entire graph
graph.delete_all()

# move content summary table
def moveContentSummaryTable():
    graph = Graph()

    lids = session.execute("SELECT DISTINCT learner_id from learnercontentsummary")
    for lid in lids:
        uid = lid['learner_id']
        print("** learner:",uid)
        # content_id text, interactions_per_min double,
        #num_of_sessions_played int,
        #time_spent double,
        node=Node("Learner",id=uid)
        graph.merge(node,"Learner","id")

        contentDict = session.execute("SELECT * from learnercontentsummary WHERE learner_id='" + uid + "'")[0]
        cid = contentDict['content_id']
        tsp = contentDict['time_spent']
        ipm = contentDict['interactions_per_min']

        node2 = graph.merge_one("Content","id",cid)
        # add a relationship with property score
        graph.create(Relationship(node, "INTERACTED_WITH", node2,timeSpent=tsp,ipm=ipm))
        print('content: ', cid, 'tsp: ',tsp, 'ipm', ipm)



# move proficiency table

def moveContentModel():
    baseURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/getContent/"
    listURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/content/list"

    # neo4j graph connector
    graph = Graph()
    
    url = listURL
    resp = requests.get(url).json()
    # no of content
    contentList = resp["result"]["contents"]
    for contentListDict in contentList:
        # check if there is an identifier for this content
        if(not contentListDict.has_key('identifier')):
            continue
    
        # check if there is an identifier for this content
        identifier = contentListDict['identifier']

        # create a node for this Content
        node = graph.merge_one("Content","id",identifier)
       
        url = baseURL + identifier
        resp = requests.get(url)

        if(resp.status_code!=200):
            continue

        resp =  resp.json()

        concept=None
        Subject=None
        ASERlevel=None
        
        contentDict = resp["result"]["content"]

    
        if(contentDict.has_key('concepts')):
            # this forms a "relationship" in the graph
            concepts = contentDict['concepts']
        
        if(contentDict.has_key('Subject')):
            # this forms a "relationship" in the graph
            Subject = contentDict['Subject']
            node.properties['Subject'] = Subject
            node.push()
            
        node.properties['tags'] = "ASERlevel_"+str(randint(1,5))
        node.push()


# move concept map 
def moveConceptMap():
    # neo4j graph connector
    graph = Graph()
    # delete entire graph

    url="http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/domain/map"
    resp = requests.get(url).json()

    # move all concepts
    conceptList = resp["result"]["concepts"]
    for conceptDict in conceptList:
        identifier=None
        ASERlevel=None
        if(not conceptDict.has_key('identifier')):
            continue

        identifier = conceptDict['identifier']
        # create/find node
        node = graph.merge_one("Concept","id",identifier)

        if(conceptDict.has_key('subject')):
            subject = conceptDict['subject']
            node.properties["subject"]=subject
            node.push()

        if(conceptDict.has_key('objectType')):
            objectType = conceptDict['objectType']
            node.properties["objectType"]=objectType
            node.push()
        node.properties['tags'] ="ASERlevel_"+str(randint(1,5))
        node.push()

        # move all relations
        relationList = resp["result"]["relations"]
    for relationDict in relationList:

        if (not relationDict.has_key('startNodeId') ):
            continue
        if (not relationDict.has_key('endNodeId') ):
            continue
        if (not relationDict.has_key('relationType') ):
            continue
        startNodeId = relationDict['startNodeId']
        endNodeId = relationDict['endNodeId']
        relationType = relationDict['relationType']
        node1 = graph.merge_one("Concept","id",startNodeId)
        node2 = graph.merge_one("Concept","id",endNodeId)
        graph.create(Relationship(node1, relationType, node2))

print('................')
moveConceptMap();
print('Concept Map populated')
# content model
print('................')
moveContentModel();
print('Concept Map populated')

................
Concept Map populated
................
Concept Map populated


In [72]:
sub="numeracy"
level="ASERlevel_1"
contentRec= graph.cypher.execute("MATCH(b:Content{Subject:'"+sub+"'}) WHERE ( has(b.tags)) AND ('"+level+"' IN b.tags ) Return b.id AS Content")
print('Recomended content:')
contentRec


Recomended content:


   | Content                      
---+-------------------------------
 1 | org.ekstep.addobj.worksheet  
 2 | org.ekstep.counting.worksheet
 3 | numeracy_382                 
 4 | org.ekstep.delta             
 5 | numeracy_418                 